In [1]:
import sys
sys.path.append("../")

In [2]:
from datasets.vocabulary import BertVocab

In [3]:
class config():
    def __init__(self):
        self.vocab_path = "../data/uncased_L-12_H-768_A-12/vocab.txt"

In [4]:
conf = config()

In [5]:
import tqdm

In [6]:
from collections import Counter

In [7]:
bert_counter = Counter()

In [8]:
print("Using Bert Vocab")
counter = Counter()
with open("../data/wikitext2/all.txt") as text_file:
    for line in tqdm.tqdm(text_file):
        if isinstance(line, list):
            words = line
        else:
            words = line.replace("\n", "").replace("\t", "").split()

        for word in words:
            counter[word] += 1

text_file.close()

Using Bert Vocab


98856it [00:01, 80945.86it/s]


In [9]:
import numpy as np

In [10]:
bert_counter = Counter()
word_list = []
with open("../data/uncased_L-12_H-768_A-12/vocab.txt") as bert_vocab:

    for word in tqdm.tqdm(bert_vocab):
        word = word.replace("\n", "").replace("\t", "")
        bert_counter[word] = counter[word] if word in counter else 1
        word_list.append(word)


30522it [00:00, 184377.62it/s]


In [11]:
import datasets

In [12]:
class config():
    def __init__(self):
        self.vocab = "bert-google"
        self.vocab_path = "../data/wikitext2/all.txt"
        self.bert_google_vocab = "../data/uncased_L-12_H-768_A-12/vocab.txt"
        self.vocab_max_size = None
        self.vocab_min_frequency = 1
        self.dataset = "wikitext2"
        self.seq_len = 40
        self.on_memory = True
        self.corpus_lines = None
        self.train_dataset = "../data/wikitext2/test_data_single_sentence.txt"
        self.encoding = "utf-8"
        self.batch_size = 1
        self.num_workers = 1

In [13]:
conf = config()

In [14]:
vocab = BertVocab(conf)

Using Bert Vocab


98856it [00:00, 121759.57it/s]
30522it [00:00, 1140365.28it/s]


In [15]:
# load the dataset specified with --dataset_name & get data loaders
train_dataset = datasets.get(dataset_name="wikitext2")(config=conf, vocab=vocab)

train_loader = train_dataset.get_data_loader()

Loading Dataset: 9961it [00:00, 15294.25it/s]


In [16]:
data = next(iter(train_loader))

In [17]:
import models

In [18]:
conf.hidden_features = 768
conf.layers = 12
conf.heads = 12
conf.device = "cpu"
conf.dropout = 0.1
conf.train = True
conf.lr = 1e-3
conf.adam_beta1=0.999
conf.adam_beta2=0.999
conf.adam_weight_decay = 0.01
conf.warmup_steps =1000

In [19]:
model = models.get(model_name="BERTLM")(config=conf, vocab_size=len(vocab))

In [20]:
for name, param in model.named_parameters():
    print(param.shape)

torch.Size([30522, 768])
torch.Size([2, 768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([3072, 768])
torch.Size([3072])
torch.Size([768, 3072])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([3072, 768])
torch.Size([3072])
torch.Size([768, 3072])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([3072, 768])
torch.Size([3072])
torch.Size([768, 3072])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Siz

In [21]:
import torch
import torch.nn as nn

### Loading TF - BERT FTW!!!

In [22]:
pt_model = torch.load("models/_checkpoints/wikitext2/BERTLM-latest.pth")

In [24]:
from copy import deepcopy

In [25]:
model_copy = deepcopy(model)

In [26]:
new_model = model.load_state_dict(pt_model['model_state_dict'])

### Comparing with Hugging Face :)

In [34]:
model.bert.embedding.token.weight

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       requires_grad=True)

In [ ]:
model

In [38]:
model.bert.embedding.position

PositionalEmbedding()

In [32]:
model.bert.embedding.position

PositionalEmbedding()

In [26]:
for name, param in model.named_parameters():
    print(name, param.shape)

bert.embedding.token.weight torch.Size([30522, 768])
bert.embedding.segment.weight torch.Size([2, 768])
bert.transformer_blocks.0.attention.linear_layers.0.weight torch.Size([768, 768])
bert.transformer_blocks.0.attention.linear_layers.0.bias torch.Size([768])
bert.transformer_blocks.0.attention.linear_layers.1.weight torch.Size([768, 768])
bert.transformer_blocks.0.attention.linear_layers.1.bias torch.Size([768])
bert.transformer_blocks.0.attention.linear_layers.2.weight torch.Size([768, 768])
bert.transformer_blocks.0.attention.linear_layers.2.bias torch.Size([768])
bert.transformer_blocks.0.attention.output_linear.weight torch.Size([768, 768])
bert.transformer_blocks.0.attention.output_linear.bias torch.Size([768])
bert.transformer_blocks.0.feed_forward.w_1.weight torch.Size([3072, 768])
bert.transformer_blocks.0.feed_forward.w_1.bias torch.Size([3072])
bert.transformer_blocks.0.feed_forward.w_2.weight torch.Size([768, 3072])
bert.transformer_blocks.0.feed_forward.w_2.bias torch.Siz

In [52]:
model.bert.embedding.token

TokenEmbedding(30522, 768, padding_idx=0)

In [47]:
model.bert.embedding.position.pe.numpy().shape

(1, 512, 768)

In [40]:
pt_model['model_state_dict']['bert.embedding.position.pe']

tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  8.2843e-01,  ...,  1.0000e+00,
           1.0243e-04,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  9.2799e-01,  ...,  1.0000e+00,
           2.0486e-04,  1.0000e+00],
         ...,
         [ 6.1950e-02,  9.9808e-01,  5.3552e-01,  ...,  9.9857e-01,
           5.2112e-02,  9.9864e-01],
         [ 8.7333e-01,  4.8714e-01,  9.9957e-01,  ...,  9.9857e-01,
           5.2214e-02,  9.9864e-01],
         [ 8.8177e-01, -4.7168e-01,  5.8417e-01,  ...,  9.9856e-01,
           5.2317e-02,  9.9863e-01]]])

In [27]:
for key, value in pt_model['model_state_dict'].items():
    print(key, value.shape)

bert.embedding.token.weight torch.Size([30522, 768])
bert.embedding.position.pe torch.Size([1, 512, 768])
bert.embedding.segment.weight torch.Size([2, 768])
bert.transformer_blocks.0.attention.linear_layers.0.weight torch.Size([768, 768])
bert.transformer_blocks.0.attention.linear_layers.0.bias torch.Size([768])
bert.transformer_blocks.0.attention.linear_layers.1.weight torch.Size([768, 768])
bert.transformer_blocks.0.attention.linear_layers.1.bias torch.Size([768])
bert.transformer_blocks.0.attention.linear_layers.2.weight torch.Size([768, 768])
bert.transformer_blocks.0.attention.linear_layers.2.bias torch.Size([768])
bert.transformer_blocks.0.attention.output_linear.weight torch.Size([768, 768])
bert.transformer_blocks.0.attention.output_linear.bias torch.Size([768])
bert.transformer_blocks.0.feed_forward.w_1.weight torch.Size([3072, 768])
bert.transformer_blocks.0.feed_forward.w_1.bias torch.Size([3072])
bert.transformer_blocks.0.feed_forward.w_2.weight torch.Size([768, 3072])
bert

In [33]:
from copy import deepcopy

In [34]:
copy_model = deepcopy(model)

In [35]:
model.load_state_dict(pt_model['model_state_dict'])

<All keys matched successfully>

In [38]:
new_preds = model(data['bert_input'], data['segment_label'])

torch.Size([1, 40, 768])
torch.Size([1, 40, 30522])


3364

In [ ]:
new_data = {'bert_input':torch.tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102,]])}

In [117]:
a = np.zeros(40)
b= np.zeros(40)
for i in range(12):
    a[i] = [ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102][i]
    b[i] = 1

In [118]:
new_data = {}

new_data['bert_input'] = torch.tensor([list(a)]).int()
new_data['segment_label'] = torch.tensor([list(b)]).int()

In [122]:
model.bert(new_data['bert_input'], new_data['segment_label'])[:, :12, :]

tensor([[[ -2.2788, -16.6172,  -7.8991,  ...,  35.9652,   1.1743,   4.1372],
         [ -7.3481,  -6.7039,  19.1383,  ...,  10.3477,   8.1672,  -1.0014],
         [ -1.1811,   0.5585,   0.0000,  ...,  41.6513,  -0.0000,  15.6352],
         ...,
         [ -4.9349,   0.6069,  26.8117,  ...,   0.0000,   0.3632,   0.3513],
         [-31.0654,   2.4595, -22.1888,  ...,   0.0000, -11.1723,  18.1958],
         [-12.5423, -16.8053,   0.0000,  ...,   6.1371, -10.0435,   6.6538]]],
       grad_fn=<SliceBackward0>)